In [11]:
import argparse
import logging
from datetime import date
import re

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col,
    current_date,
    current_timestamp,
    lit,
    when,
    concat_ws,
    expr,
    date_format,
)
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable


LOG = logging.getLogger("medallion")

def clean_name(c: str) -> str:
    c = c.strip()
    c = re.sub(r"[;:{}()\n\t\r]", "", c)
    c = re.sub(r"\s+", "_", c)
    c = re.sub(r"[^A-Za-z0-9_]", "", c)
    return c


def build_spark(app_name="medallion_pipeline"):
    builder = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .config("spark.executor.memory", "1g")
    )
    return configure_spark_with_delta_pip(builder).getOrCreate()


def ingest_to_bronze(spark, input_path, bronze_path):
    LOG.info("Leyendo archivo de entrada: %s", input_path)
    df_raw = (
        spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load(input_path)
    )

In [12]:
    # Limpiar nombres de columnas (coincidir con el resto del pipeline)
    cols_limpias = [clean_name(c) for c in df_raw.columns]
    for old, new in zip(df_raw.columns, cols_limpias):
        if old != new:
            df_raw = df_raw.withColumnRenamed(old, new)

NameError: name 'df_raw' is not defined